### Tutorial: Optimal Strategy Given a Fixed Observing Budget

In this simplest example, let's say you have a budget of 60 observations on a spectrograph, and you want to spend them on a single target. How do you best allocate your budget to maximize the Fisher Information (or minimize the uncertainty) on the radial velocity semi-amplitude, K?

For this tutorial, we're going to anonymize the actual first usecase of gaspery, while remaining as faithful to the process as possible. 

In [5]:
import numpy as np 
import scipy
print(np.__version__)
print(scipy.__version__)
import pandas as pd
import random
import exoplanet
import astropy 
import pymc3
import pymc3_ext
import celerite2
from numpy.linalg import inv, det, solve, cond
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
#from jax import random

from gaspery import calculate_fi, strategies, utils



1.22.3
1.7.3


In [6]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(params)

path = '/Users/chrislam/Desktop/gaspery/'

In [7]:
### Time to prep ### 

### observing parameters
sigma_ks = []
c = 1 # observing cadence of once per day
start = 2352
n_obs = 200

### target parameters
sigma_wn_rv = 500 # cm/s (from Klein+ 2020)
p = 8.5 # orbital period, days
Prot = 4.86 # rotation period, days
K = 580 # cm/s
T0 = 2360 # arbitrarily chosen fiducial central transit time, in BJD

### correlated noise parameters, from Klein+ 2021 for AU Mic
Tau = 100/np.sqrt(2) # active region lifetime; days
eta = 0.4/np.sqrt(2) # 0.1, 0.3, 0.58, 0.9 # smoothing parameter
sigma_qp_rv = 47 * 1e2 # modified Jeffreys prior +11, -8 [cm/s]
sigma_wn_rv = 5 * 1e2 # [cm/s]

params = [Tau, eta, Prot, sigma_qp_rv, sigma_wn_rv]
theta = [K, p, T0]

In [8]:
### Time to start cooking ###
